# Simple ETL for the forceplate data

Before we are able to transform the data it is necessary to load important packages and libraries.

In [ ]:
import os
from pathlib import Path
from nptdms import TdmsFile as td

import pixiedust
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.sql.functions import input_file_name, lit

## Transform native data to csv files

Data coming from the forceplate sensor is stored in a proprietary format that Spark cannot read directly. 
Thus we need to extract the data and store them in an open fromat, as CSV.

Forceplate data are stored in files with the extention _.tdms_. under a top folder named `files`.  
First we generate a list containing the files in the appropriate folder with the extention _.tdms_.
Secondly, we iterate through the list and transform the native data to _.csv_
format and store them in another folder, under _work_ top folder.

In [ ]:
pathlist = Path("files").glob('**/*.tdms')
for filename in pathlist:
    print(filename)
    directory = os.path.dirname(filename)
    destination = directory + '.csv'
    td(filename).as_dataframe().to_csv('work/forceplate/original/'+os.path.basename(destination))

## Read the created .csv file

To verify if Spark is running in your note, type `spark`.

Discuss in classroom what the response is.

In [ ]:
spark

Had you have a look in the `tdms` files, you notice that the column names are way too long, thus we will define manually the schema into which Spark will load them. 


In the schema declaratiob below, we define the column names, and the corresponding data types.

In [ ]:
schema = StructType([
    StructField("Time", IntegerType()),
    StructField("Channel1", DoubleType()),
    StructField("Channel2", DoubleType()),
    StructField("Channel3", DoubleType()),
    StructField("Channel4", DoubleType()),
    StructField("Channel5", DoubleType()),
    StructField("Channel6", DoubleType()),
    StructField("Channel7", DoubleType()),
    StructField("Channel8", DoubleType())
])

Then, lets read the _.csv_ file, with the schema we set-up earlier.

In [ ]:
channelsDF = spark.read.csv('work/forceplate/original/18936.csv', header=True, schema=schema)

Then, lets inspect the dataframe and its schema.

In [ ]:
channelsDF.schema

How many columns are there in the dataframe? Run the cell below!

In [ ]:
len(channelsDF.columns)

Use the `count` action to see how many data points are in the dataframe

In [ ]:
channelsDF.count()

To calculate summary statistics from the dataframe, use the function `describe`.

In [ ]:
channelsDF.describe().show()

If you want to know what Spark is doing behind the hood, the `explain` function prints the physical plan.

In [ ]:
channelsDF.explain()

With `select` you can select one or more columns, and chain it with `show` to inspect it.

In [ ]:
channelsDF.select('Channel1').show(2)

Keep on exploring the data you have just loaded.

In [ ]:
channelsDF

channelsDF.show(5)

## Calculate  fixed parameters for the Force Plate

Next step is to calculate the forces on the plate.
![image](http://c-motion.com/v3dwiki/images/b/b1/fp_type6.jpg)
The calculations below are from 
[Kistler](https://isbweb.org/software/movanal/vaughan/kistler.pdf) sensor documentation,  
including calculations for:

- Fx = Medio-lateral force 
- Fy = Anterior-posterior force
- Fz = Vertical force
- ax = X-Coordinate of force application point 
- ay =Y-Coordinate of force application point

Constant values for the equations are defined below.

In [ ]:
amp2 = 1.5  # --- amplitude-parameter, van belang voor de eerste grafiek ---
is1 = 2
ixy = 5000  # --- versterker-schaal x/y
is2 = 2
iz = 5000   # --- versterker-schaal z

Then the $F_{x}$ is calculated as:

In [ ]:
df = channelsDF.withColumn("Fx",(channelsDF.Channel1+channelsDF.Channel2)*ixy/76.33)

Note that we can chain up all these computations as:

In [ ]:
df = channelsDF.withColumn("Fx",(channelsDF.Channel1+channelsDF.Channel2)*ixy/76.33)\
               .withColumn("Fy",(channelsDF.Channel3+channelsDF.Channel4)*ixy/75.92)\
               .withColumn("Fz",((channelsDF.Channel5+channelsDF.Channel6+channelsDF.Channel7+channelsDF.Channel8)*iz)/38.65)\
               .withColumn("Fz1",(channelsDF.Channel5)*iz/38.65)\
               .withColumn("Fz2",(channelsDF.Channel6)*iz/38.65)\
               .withColumn("Fz3",(channelsDF.Channel7)*iz/38.65)\
               .withColumn("Fz4",(channelsDF.Channel8)*iz/38.65)

And keep on with some more as follows.

In [ ]:
df = df.withColumn("Mx",(200*(df.Fz1+df.Fz2-df.Fz3-df.Fz4)))\
                    .withColumn("My",(120*(df.Fz2+df.Fz3-df.Fz1-df.Fz4)))\
                    #.withColumn("Dz",((channelsDF.Channel5+channelsDF.Channel6+channelsDF.Channel7+channelsDF.Channel8)*iz/38.65[0:50].mean().first()[0]))\

In [ ]:
df = df.withColumn("Mx1", (df.Mx + df.Fy*-45.))\
         .withColumn("My1", (df.My - df.Fx*-45.))\

In [ ]:
df = df.withColumn("ax", (-df.My1 / df.Fz))\
         .withColumn("ay", (df.Mx1 / df.Fz))\

In [ ]:
df.show(5)

## Explore visually your data 

Now that all forces and quantities have been calculated, we will use pixiedust to visually explore our data. 

We use the command `display("yourdf")` to open the pixiedust GUI in the Jupyter Notebook.


Lets start with creating a scatterplot. Select

- X-axis (i.e. keys) = *Time*, 
- Y-axis (i.e. values) = *$F_z$*,

and set the number of rows to 6000.

In [ ]:
display(df)

## Summarize and extract (some) features

To check if everything is right, print summary statistics

In [ ]:
df.select(df.Fx,df.Fy).describe().show()
df.select(df.Fz, df.Fz1, df.Fz2, df.Fz3, df.Fz4).describe().show()
df.select(df.Mx,df.Mx1, df.My, df.My1).describe().show()
df.select(df.ax,df.ay).describe().show()

To calculate the maximum $F_z$ value, we will use an aggregation function `max`. 
Note that an aggregation function is always combined with a `groupBy` function.

In [ ]:
df.select(df.Fz).groupBy().max().first()[0]

$F_z$ = m * g, thus m = $F_z$ / g, where g is Newton earth (9.80665)

In [ ]:
print("The turkey weighs approximately", round(df.select(df.Fz).groupBy().max().first()[0] / 9.80665,2), "kg")

## Save the file for future use


We are finished with preprocessing. Lets store the pre-processed dataset in a
CSV format.


First select the columns and then store:

In [ ]:
df_updated = df.select(df.Time, df.Fx, df.Fy, df.Fz, df.Fz1, df.Fz2, df.Fz3, df.Fz4, df.Mx, df.Mx1, df.My, df.My1)
df_updated.write.csv('work/forceplate/updated/18936.csv', header=True, mode='overwrite')

Wonderful! Check the newly created file [in the 'work/forceplate/updated'
folder](../tree/work/forceplate/updated).

Question: What do you observe?


## Extra visualizations

Try to focus your diagram on when the turkey is walking on the forceplate.

Assuming that all vertical forces below 100 are noize, visualize again $F_z$, as a scatterplot with `Time` on the X-axis and `Fz` on the Y-axis.

In [ ]:
df_updated2 = df_updated.filter(df_updated.Fz>100)

display(df_updated2)

Discuss in pairs what features could be extracted, that can help understand the gait score of turkeys.

If time allows, play around with different visualizations.
As an example create a scatterplot that shows the coordinates of the center of pressure coordinates (`ax`, `ay`)

In [ ]:
display(df) #scattered e.g. ax vs. ay